In [19]:
# 필요한 모듈 및 라이브러리 불러오기
from selenium.webdriver import Chrome
import time
import sqlite3
from pandas.io import sql
import os
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import random
import sys
import re
import requests
from bs4 import BeautifulSoup
import datetime
import os
import sqlite3
import csv
import warnings
warnings.filterwarnings(action='ignore')

In [20]:
# 필요한 데이터 생성
customer_allergy= pd.read_csv('/Users/JGH/Python/Section Project 4/customer_allergy.csv')
drink_allergy= pd.read_csv('/Users/JGH/Python/Section Project 4/drink_allergy.csv')
allergy_list = list(set(customer_allergy.allergy.unique().tolist() + drink_allergy.allergy.unique().tolist()))

In [21]:
DB_FILENAME = 'project4.db'
DB_FILEPATH = os.path.join(os.getcwd(), DB_FILENAME)

conn = sqlite3.connect(DB_FILENAME)
cur = conn.cursor()

# 테이블 생성

In [22]:
# 고객 테이블 
cur.execute("""
CREATE TABLE CUSTOMER (
	id 		Int 			NOT NULL PRIMARY KEY,
	sex		Varchar(45)		NOT NULL,
	age		Varchar(45)		NOT NULL
    );
""")

# 음료 테이블
cur.execute("""
CREATE TABLE DRINK (
	id 		        Int			    NOT NULL PRIMARY KEY,
	name		    Varchar(45)		NOT NULL,
	category		Varchar(45)		NOT NULL,
    price           Int             NOT NULL,
    calories        Int             NOT NULL,
    sugars          Int             NOT NULL,
    protein         Int             NOT NULL,
    sodium          Int             NOT NULL,
    saturated_fat   Int             NOT NULL,
    caffeine        Int             NOT NULL,
    status          Varchar(45)     NOT NULL
    );
""")

# 알레르기 테이블
cur.execute("""
CREATE TABLE ALLERGY (
	allergy		    Varchar(45)		NOT NULL PRIMARY KEY);
""")

# 고객-알레르기 테이블
cur.execute("""
CREATE TABLE C_Allergy (
    id      Int             NOT NULL,
    allergy Varchar(45)     NOT NULL,
    PRIMARY KEY (id, allergy),
    FOREIGN KEY (id)
        REFERENCES CUSTOMER(id)
        ON UPDATE CASCADE
        ON DELETE CASCADE,
    FOREIGN KEY (allergy)
        REFERENCES ALLERGY(allergy)
        ON UPDATE CASCADE
        ON DELETE CASCADE        
);
""")

# 음료-알레르기 테이블
cur.execute("""
CREATE TABLE D_Allergy (
    id      Int             NOT NULL,
    name    Varchar(45)     NOT NULL,
    allergy Varchar(45)     NOT NULL,
    PRIMARY KEY (id, allergy),
    FOREIGN KEY (id)
        REFERENCES DRINK(id)
        ON UPDATE CASCADE
        ON DELETE CASCADE,
    FOREIGN KEY (name)
        REFERENCES DRINK(name)
        ON UPDATE CASCADE
        ON DELETE CASCADE,
    FOREIGN KEY (allergy)
        REFERENCES ALLERGY(allergy)
        ON UPDATE CASCADE
        ON DELETE CASCADE        
);
""")

# 주문 테이블
cur.execute("""
CREATE TABLE ORDER_DB(
	id 		        Int		       NOT NULL PRIMARY KEY,
	customer_id		Int		       NOT NULL,
	drink_id		Int		       NOT NULL,
    drink_name      Varchar(45)    NOT NULL,
    date            TEXT           NOT NULL,
    FOREIGN KEY (customer_id)
        REFERENCES CUSTOMER(id)
        ON UPDATE CASCADE
        ON DELETE CASCADE,
    FOREIGN KEY (drink_id)
        REFERENCES DRINK(id)
        ON UPDATE CASCADE
        ON DELETE CASCADE,
    FOREIGN KEY (drink_name)
        REFERENCES DRINK(name)
        ON UPDATE CASCADE
        ON DELETE CASCADE         
    );
""")

# 데이터 입력

In [23]:
# 고객 테이블
with open('/Users/JGH/Python/Section Project 4/customer_db.csv', 'r') as file:
    csv_data = csv.reader(file)
    next(csv_data)  # 헤더 행 스킵
    cur.executemany("INSERT INTO CUSTOMER VALUES (?, ?, ?)", csv_data)

# 음료 테이블
with open('/Users/JGH/Python/Section Project 4/drink_db.csv', 'r') as file:
    csv_data = csv.reader(file)
    next(csv_data)  # 헤더 행 스킵
    cur.executemany("INSERT INTO DRINK VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", csv_data)

# 알레르기 테이블
for allergy in allergy_list:
    cur.execute("INSERT INTO ALLERGY (allergy) VALUES (?)", (allergy,))

# 고객-알레르기 테이블
with open('/Users/JGH/Python/Section Project 4/customer_allergy.csv', 'r') as file:
    csv_data = csv.reader(file)
    next(csv_data)  # 헤더 행 스킵
    cur.executemany("INSERT INTO C_ALLERGY VALUES (?, ?)", csv_data)
    
# 음료-알레르기 테이블
with open('/Users/JGH/Python/Section Project 4/drink_allergy.csv', 'r') as file:
    csv_data = csv.reader(file)
    next(csv_data)  # 헤더 행 스킵
    cur.executemany("INSERT INTO D_ALLERGY VALUES (?, ?, ?)", csv_data)
    
# 주문 테이블
with open('/Users/JGH/Python/Section Project 4/order_db.csv', 'r') as file:
    csv_data = csv.reader(file)
    next(csv_data)  # 헤더 행 스킵
    cur.executemany("INSERT INTO ORDER_DB VALUES (?, ?, ?, ?, ?)", csv_data)

In [24]:
conn.commit()
conn.close()